# Подключение зависимостей

In [262]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ccxt
from IPython.display import clear_output, display
import dill

#sns.set_theme()
%matplotlib inline

In [3]:
import mlflow
from mlflow.tracking import MlflowClient
import mlflavors
import mlforecast.flavor

from statsforecast import StatsForecast
from statsforecast.models import ARCH, GARCH
from mlforecast import MLForecast
from mlforecast.utils import PredictionIntervals
from datasetsforecast import losses
from utilsforecast.plotting import plot_series

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import lightgbm as lgb

# Подключение к MLFlow

In [4]:
mlflow.set_tracking_uri("http://mlflow:5000")

In [5]:
mlflow.set_experiment("btc-usdt_volatility_experiment")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1727343300546, experiment_id='2', last_update_time=1727343300546, lifecycle_stage='active', name='btc-usdt_volatility_experiment', tags={}>

# Проведение экспериментов 

## Подготовка данных

In [211]:
# Биржа из которой будут браться данные с помощью CCXT
EXCHANGE = ccxt.okx()
# Инструмент в формате символа для обработки
SYMBOL = "BTC/USDT"
# Таймфрейм свеч
TIMEFRAME = "1h"

In [212]:
from_ts = EXCHANGE.parse8601('2024-01-10 00:00:00')

ohlcv_list = []
ohlcv = EXCHANGE.fetch_ohlcv(symbol=SYMBOL, timeframe=TIMEFRAME, since=from_ts, limit=100)
ohlcv_list.append(ohlcv)

while True:
    from_ts = ohlcv[-1][0]
    new_ohlcv = EXCHANGE.fetch_ohlcv(symbol=SYMBOL, timeframe=TIMEFRAME, since=from_ts, limit=100)
    ohlcv.extend(new_ohlcv)

    print(f"\r{EXCHANGE.iso8601(from_ts)}", end="")
    
    if len(new_ohlcv) <= 1:
    	break

2024-09-27T11:00:00.000Z

In [213]:
ohlcv = pd.DataFrame(ohlcv, columns=["date", "open", "high", "low", "close", "volume"])
ohlcv["date"] = ohlcv["date"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))
ohlcv["close_pct_change"] = ohlcv["close"].pct_change()
ohlcv['close_pct_change'] = ohlcv['close_pct_change'].fillna(0)
ohlcv.reset_index(drop=True)

,date,open,high,low,close,volume,close_pct_change
0,2024-01-10 00:00:00,46109.4,46235.5,45768.3,45854.1,1198.592272,0.000000
1,2024-01-10 01:00:00,45853.9,45955.6,45617.5,45925.8,1463.003867,0.001564
2,2024-01-10 02:00:00,45925.8,46083.0,45878.2,45970.8,640.245350,0.000980
3,2024-01-10 03:00:00,45978.0,46209.9,45923.3,46119.9,359.281616,0.003243
4,2024-01-10 04:00:00,46119.9,46180.9,45941.2,45961.9,380.550457,-0.003426
...,...,...,...,...,...,...,...
6335,2024-09-27 08:00:00,65464.7,65798.9,65417.0,65731.9,340.863833,0.004082
6336,2024-09-27 09:00:00,65732.0,65997.2,65662.8,65746.8,474.897545,0.000227
6337,2024-09-27 10:00:00,65746.3,65781.1,65488.0,65488.0,249.032124,-0.003936
6338,2024-09-27 11:00:00,65488.1,65616.3,65342.0,65389.5,142.224558,-0.001504


In [214]:
ohlcv

,date,open,high,low,close,volume,close_pct_change
0,2024-01-10 00:00:00,46109.4,46235.5,45768.3,45854.1,1198.592272,0.000000
1,2024-01-10 01:00:00,45853.9,45955.6,45617.5,45925.8,1463.003867,0.001564
2,2024-01-10 02:00:00,45925.8,46083.0,45878.2,45970.8,640.245350,0.000980
3,2024-01-10 03:00:00,45978.0,46209.9,45923.3,46119.9,359.281616,0.003243
4,2024-01-10 04:00:00,46119.9,46180.9,45941.2,45961.9,380.550457,-0.003426
...,...,...,...,...,...,...,...
6335,2024-09-27 08:00:00,65464.7,65798.9,65417.0,65731.9,340.863833,0.004082
6336,2024-09-27 09:00:00,65732.0,65997.2,65662.8,65746.8,474.897545,0.000227
6337,2024-09-27 10:00:00,65746.3,65781.1,65488.0,65488.0,249.032124,-0.003936
6338,2024-09-27 11:00:00,65488.1,65616.3,65342.0,65389.5,142.224558,-0.001504


In [215]:
ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6340 entries, 0 to 6339
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              6340 non-null   datetime64[ns]
 1   open              6340 non-null   float64       
 2   high              6340 non-null   float64       
 3   low               6340 non-null   float64       
 4   close             6340 non-null   float64       
 5   volume            6340 non-null   float64       
 6   close_pct_change  6340 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 346.8 KB


In [216]:
ohlcv.shape

(6340, 7)

In [217]:
ohlcv.nunique()

date                6276
open                6132
high                6023
low                 5997
close               6120
volume              6277
close_pct_change    6275
dtype: int64

In [221]:
ohlcv["date"].astype(int)

0       1704844800000000000
1       1704848400000000000
2       1704852000000000000
3       1704855600000000000
4       1704859200000000000
               ...         
6335    1727424000000000000
6336    1727427600000000000
6337    1727431200000000000
6338    1727434800000000000
6339    1727434800000000000
Name: date, Length: 6340, dtype: int64

В данных могут появится дубликаты, от них надо избавлятся

In [224]:
ohlcv = ohlcv.drop_duplicates(subset=['date'], keep='last')

In [225]:
ohlcv

,date,open,high,low,close,volume,close_pct_change
0,2024-01-10 00:00:00,46109.4,46235.5,45768.3,45854.1,1198.592272,0.000000
1,2024-01-10 01:00:00,45853.9,45955.6,45617.5,45925.8,1463.003867,0.001564
2,2024-01-10 02:00:00,45925.8,46083.0,45878.2,45970.8,640.245350,0.000980
3,2024-01-10 03:00:00,45978.0,46209.9,45923.3,46119.9,359.281616,0.003243
4,2024-01-10 04:00:00,46119.9,46180.9,45941.2,45961.9,380.550457,-0.003426
...,...,...,...,...,...,...,...
6334,2024-09-27 07:00:00,65344.0,65546.5,65260.0,65464.7,592.119167,0.001849
6335,2024-09-27 08:00:00,65464.7,65798.9,65417.0,65731.9,340.863833,0.004082
6336,2024-09-27 09:00:00,65732.0,65997.2,65662.8,65746.8,474.897545,0.000227
6337,2024-09-27 10:00:00,65746.3,65781.1,65488.0,65488.0,249.032124,-0.003936


In [226]:
train_df = pd.DataFrame(
    columns = ["ds", "y", "unique_id"]   
)

train_df["ds"] = ohlcv["date"].iloc[-324:-24]
train_df["y"] = ohlcv["close_pct_change"].iloc[-324:-24]
train_df["unique_id"] = 1
train_df = train_df.reset_index(drop=True)

In [227]:
train_df

,ds,y,unique_id
0,2024-09-14 00:00:00,-0.001550,1
1,2024-09-14 01:00:00,-0.000061,1
2,2024-09-14 02:00:00,-0.001869,1
3,2024-09-14 03:00:00,0.000650,1
4,2024-09-14 04:00:00,-0.001765,1
...,...,...,...
295,2024-09-26 07:00:00,0.002040,1
296,2024-09-26 08:00:00,-0.002106,1
297,2024-09-26 09:00:00,0.001256,1
298,2024-09-26 10:00:00,0.009356,1


In [228]:
test_df = pd.DataFrame(
    columns = ["ds", "y", "unique_id"]   
)

test_df["ds"] = ohlcv["date"].iloc[-24:]
test_df["y"] = ohlcv["close_pct_change"].iloc[-24:]
test_df["unique_id"] = 1
test_df = test_df.reset_index(drop=True)

In [229]:
test_df

,ds,y,unique_id
0,2024-09-26 12:00:00,0.001298,1
1,2024-09-26 13:00:00,-0.000771,1
2,2024-09-26 14:00:00,-0.000045,1
3,2024-09-26 15:00:00,0.012440,1
4,2024-09-26 16:00:00,0.001646,1
5,2024-09-26 17:00:00,-0.003295,1
6,2024-09-26 18:00:00,0.002441,1
7,2024-09-26 19:00:00,-0.008975,1
8,2024-09-26 20:00:00,-0.000745,1
9,2024-09-26 21:00:00,0.006093,1


## MLFlow run

### ARCH

In [251]:
def make_arch_run(**kwargs):    
    with mlflow.start_run(run_name=f'ARCH_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "p": kwargs["p"], 
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "ARCH")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = StatsForecast(
            models=[ARCH(**params)],
            freq='h',
            n_jobs=-1
        )

        # Обучение моедли
        model.fit(train_df)

        # Прогнозирование для test датасета
        forecasts = model.forecast(48, level=[95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlflavors.statsforecast.log_model(
            statsforecast_model=model,
            artifact_path="arch",
            serialization_format="pickle",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [252]:
make_arch_run(
    p=84
)

/usr/local/lib/python3.11/site-packages/statsforecast/core.py:510: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/statsforecast/core.py:528: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
2024/09/27 12:18:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp0qk8tv5b/model/model.pkl, flavor: statsforecast). Fall back to return ['statsforecast==1.7.7.1']. Set logging level to DEBUG to see the full traceback. 
2024/09/27 12:18:40 WARNING mlflow.models.model: Model logged without a signature an

### GARCH

In [253]:
def make_garch_run(**kwargs):    
    with mlflow.start_run(run_name=f'GARCH_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "p": kwargs["p"], 
            "q": kwargs["q"]
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "GARCH")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = StatsForecast(
            models=[GARCH(**params)],
            freq='h',
            n_jobs=-1
        )

        # Обучение моедли
        model.fit(train_df)

        # Прогнозирование для test датасета
        forecasts = model.forecast(48, level=[95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlflavors.statsforecast.log_model(
            statsforecast_model=model,
            artifact_path="garch",
            serialization_format="pickle",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [254]:
make_garch_run(
    p=92, 
    q=24
)

/usr/local/lib/python3.11/site-packages/statsforecast/core.py:510: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/statsforecast/core.py:528: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
2024/09/27 12:18:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpk4dv5ffu/model/model.pkl, flavor: statsforecast). Fall back to return ['statsforecast==1.7.7.1']. Set logging level to DEBUG to see the full traceback. 
2024/09/27 12:18:43 WARNING mlflow.models.model: Model logged without a signature an

### SVR

In [255]:
def make_svr_run(**kwargs):    
    with mlflow.start_run(run_name=f'SVR_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "kernel": kwargs["kernel"], 
            "degree": kwargs["degree"],
            "C": kwargs["C"]
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "SVR")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = MLForecast(
            models=[SVR(**params)],
            freq='h',  
            lags=list(range(1, 24, 1)),
        )
        
        # Обучение моедли
        model.fit(train_df, prediction_intervals=PredictionIntervals(n_windows=2, h=48))

        # Прогнозирование для test датасета
        #forecasts = model.predict(24, new_df=test_df)
        forecasts = model.predict(48, level=[95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlforecast.flavor.log_model(
            model=model,
            artifact_path="scr",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [256]:
make_svr_run(
    kernel="poly", 
    degree=3,
    C=0.9702637495163653
)

2024/09/27 12:18:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/27 12:18:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run SVR_2024-09-27 12:18:44.879510 at: http://mlflow:5000/#/experiments/2/runs/4172747f469b4baea2e56ebb92f59201.
2024/09/27 12:18:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/2.


### LGBMRegressor

In [257]:
def make_lightgbm_run(**kwargs):    
    with mlflow.start_run(run_name=f'LightGBM_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "n_estimators": kwargs["n_estimators"], 
            "boosting_type": kwargs["boosting_type"],
            "num_leaves": kwargs["num_leaves"],
            "max_depth": kwargs["max_depth"],
            "learning_rate": kwargs["learning_rate"],
            "verbose": -1
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "LightGBM")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = MLForecast(
            models=[lgb.LGBMRegressor(**params)],
            freq='h',  
            lags=list(range(1, 17, 1)),
        )

        # Обучение моедли
        model.fit(train_df, prediction_intervals=PredictionIntervals(n_windows=2, h=48))

        # Прогнозирование для test датасета
        #forecasts = model.predict(24, new_df=test_df)
        forecasts = model.predict(48, level=[95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlforecast.flavor.log_model(
            model=model,
            artifact_path="lightgbm",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [258]:
make_lightgbm_run(
    n_estimators=382, 
    boosting_type="dart", 
    num_leaves=93, 
    max_depth=7, 
    learning_rate=0.04001572844964948
)

2024/09/27 12:18:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/27 12:18:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_2024-09-27 12:18:50.295472 at: http://mlflow:5000/#/experiments/2/runs/175de804e21e4f97b573b1b068269eae.
2024/09/27 12:18:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/2.


### KNN

In [259]:
def make_knn_run(**kwargs):    
    with mlflow.start_run(run_name=f'KNN_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "n_neighbors": kwargs["n_neighbors"], 
            "weights": kwargs["weights"],
            "leaf_size": kwargs["leaf_size"]
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "KNN")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = MLForecast(
            models=[KNeighborsRegressor(**params)],
            freq='h',  
            lags=list(range(1, 24, 1)),
        )

        # Обучение моедли
        model.fit(train_df, prediction_intervals=PredictionIntervals(n_windows=2, h=48))

        # Прогнозирование для test датасета
        #forecasts = model.predict(24, new_df=test_df)
        forecasts = model.predict(48, level=[95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlforecast.flavor.log_model(
            model=model,
            artifact_path="knn",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [260]:
make_knn_run(
    n_neighbors=27, 
    weights='uniform', 
    leaf_size=34
)

2024/09/27 12:19:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/27 12:19:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run KNN_2024-09-27 12:18:55.937902 at: http://mlflow:5000/#/experiments/2/runs/3ad148774ff7465593a535d03866ae24.
2024/09/27 12:19:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/2.


# P.s.

MLFlow load model не работает

In [279]:
import requests

In [311]:
response = requests.get("http://backend:8000/api/forecast/arch/")

data = None

# Проверка статуса ответа
if response.status_code == 200:
    # Преобразование ответа в формат JSON
    data = response.json()
else:
    pass

df_now = pd.DataFrame(data["now"])

df_forecast = pd.DataFrame(data["forecast"])

df_now["ds"] = pd.to_datetime(df_now['ds'])
df_forecast["ds"] = pd.to_datetime(df_forecast['ds'])

df_now["ds"] += pd.Timedelta(hours=3)
df_forecast["ds"] += pd.Timedelta(hours=3)

print(df_now)
print(df_forecast)


response = requests.get("http://backend:8000/api/get_btc_usdt_data/")

data = None

# Проверка статуса ответа
if response.status_code == 200:
    # Преобразование ответа в формат JSON
    data = response.json()
else:
    pass

df_btc_usdt = pd.DataFrame(data["result"])
df_btc_usdt = df_btc_usdt.rename(columns={"date": "ds"})

df_btc_usdt["ds"] = pd.to_datetime(df_btc_usdt['ds'])

df_btc_usdt["ds"] += pd.Timedelta(hours=3)

print(df_btc_usdt)

                           ds  close_pct_change  unique_id
0   2024-09-15 11:00:00+00:00         -0.192711          1
1   2024-09-15 12:00:00+00:00         -0.100536          1
2   2024-09-15 13:00:00+00:00         -0.045653          1
3   2024-09-15 14:00:00+00:00          0.030338          1
4   2024-09-15 15:00:00+00:00          0.165311          1
..                        ...               ...        ...
295 2024-09-27 18:00:00+00:00         -0.367883          1
296 2024-09-27 19:00:00+00:00          0.454242          1
297 2024-09-27 20:00:00+00:00         -0.573153          1
298 2024-09-27 21:00:00+00:00         -0.066748          1
299 2024-09-27 22:00:00+00:00         -0.017912          1

[300 rows x 3 columns]
                          ds  ARCH(84)  ARCH(84)-lo-95  ARCH(84)-lo-90  \
0  2024-09-27 22:00:00+00:00  0.816481       -0.793533       -0.534685   
1  2024-09-27 23:00:00+00:00 -0.625765       -2.267501       -2.003553   
2  2024-09-28 00:00:00+00:00 -0.112889       -

In [312]:
df_now

,ds,close_pct_change,unique_id
0,2024-09-15 11:00:00+00:00,-0.192711,1
1,2024-09-15 12:00:00+00:00,-0.100536,1
2,2024-09-15 13:00:00+00:00,-0.045653,1
3,2024-09-15 14:00:00+00:00,0.030338,1
4,2024-09-15 15:00:00+00:00,0.165311,1
...,...,...,...
295,2024-09-27 18:00:00+00:00,-0.367883,1
296,2024-09-27 19:00:00+00:00,0.454242,1
297,2024-09-27 20:00:00+00:00,-0.573153,1
298,2024-09-27 21:00:00+00:00,-0.066748,1


In [313]:
df_forecast

,ds,ARCH(84),ARCH(84)-lo-95,ARCH(84)-lo-90,ARCH(84)-hi-90,ARCH(84)-hi-95,unique_id
0,2024-09-27 22:00:00+00:00,0.816481,-0.793533,-0.534685,2.167647,2.426495,1
1,2024-09-27 23:00:00+00:00,-0.625765,-2.267501,-2.003553,0.752024,1.015972,1
2,2024-09-28 00:00:00+00:00,-0.112889,-1.766803,-1.500897,1.275120,1.541026,1
3,2024-09-28 01:00:00+00:00,-0.855640,-2.494312,-2.230856,0.519577,0.783033,1
4,2024-09-28 02:00:00+00:00,-0.248372,-1.933159,-1.662290,1.165546,1.436416,1
5,2024-09-28 03:00:00+00:00,0.492230,-1.196072,-0.924637,1.909097,2.180532,1
6,2024-09-28 04:00:00+00:00,-0.644820,-2.348310,-2.074434,0.784793,1.058669,1
7,2024-09-28 05:00:00+00:00,-0.372636,-2.101562,-1.823597,1.078325,1.356290,1
8,2024-09-28 06:00:00+00:00,-0.643863,-2.364553,-2.087912,0.800186,1.076828,1
9,2024-09-28 07:00:00+00:00,-0.797885,-2.544529,-2.263715,0.667944,0.948759,1


In [314]:
df_btc_usdt

,ds,close,close_pct_change,high,low,open,volume
0,2024-09-15 11:00:00+00:00,60077.8,-0.192711,60205.3,60038.0,60193.7,98.328884
1,2024-09-15 12:00:00+00:00,60017.4,-0.100536,60109.5,60008.1,60075.1,82.532530
2,2024-09-15 13:00:00+00:00,59990.0,-0.045653,60056.0,59876.1,60017.4,70.003297
3,2024-09-15 14:00:00+00:00,60008.2,0.030338,60044.0,59900.2,59990.0,76.549696
4,2024-09-15 15:00:00+00:00,60107.4,0.165311,60132.9,59953.1,60008.2,76.634213
...,...,...,...,...,...,...,...
295,2024-09-27 18:00:00+00:00,66000.1,-0.367883,66425.6,66000.1,66243.8,489.643880
296,2024-09-27 19:00:00+00:00,66299.9,0.454242,66364.0,65893.3,66000.0,429.529991
297,2024-09-27 20:00:00+00:00,65919.9,-0.573153,66355.1,65676.7,66299.9,447.202219
298,2024-09-27 21:00:00+00:00,65875.9,-0.066748,65935.6,65520.1,65920.0,280.565846


In [315]:
df_btc_usdt

,ds,close,close_pct_change,high,low,open,volume
0,2024-09-15 11:00:00+00:00,60077.8,-0.192711,60205.3,60038.0,60193.7,98.328884
1,2024-09-15 12:00:00+00:00,60017.4,-0.100536,60109.5,60008.1,60075.1,82.532530
2,2024-09-15 13:00:00+00:00,59990.0,-0.045653,60056.0,59876.1,60017.4,70.003297
3,2024-09-15 14:00:00+00:00,60008.2,0.030338,60044.0,59900.2,59990.0,76.549696
4,2024-09-15 15:00:00+00:00,60107.4,0.165311,60132.9,59953.1,60008.2,76.634213
...,...,...,...,...,...,...,...
295,2024-09-27 18:00:00+00:00,66000.1,-0.367883,66425.6,66000.1,66243.8,489.643880
296,2024-09-27 19:00:00+00:00,66299.9,0.454242,66364.0,65893.3,66000.0,429.529991
297,2024-09-27 20:00:00+00:00,65919.9,-0.573153,66355.1,65676.7,66299.9,447.202219
298,2024-09-27 21:00:00+00:00,65875.9,-0.066748,65935.6,65520.1,65920.0,280.565846


In [316]:
last_close_price = df_btc_usdt["close"].iloc[-1]
last_close_price

65864.1

In [317]:
df_forecast_new = df_forecast.copy()

In [318]:
df_forecast_new.columns

Index(['ds', 'ARCH(84)', 'ARCH(84)-lo-95', 'ARCH(84)-lo-90', 'ARCH(84)-hi-90',
       'ARCH(84)-hi-95', 'unique_id'],
      dtype='object')

In [325]:
df_forecast_new['ARCH(84)'] = last_close_price * (1 + df_forecast_new['ARCH(84)'] / 100).cumprod()
df_forecast_new['ARCH(84)-lo-95'] = last_close_price * (1 + df_forecast_new['ARCH(84)-lo-95'] / 100).cumprod()
df_forecast_new['ARCH(84)-lo-90'] = last_close_price * (1 + df_forecast_new['ARCH(84)-lo-90'] / 100).cumprod() 
df_forecast_new['ARCH(84)-hi-90'] = last_close_price * (1 + df_forecast_new['ARCH(84)-hi-90'] / 100).cumprod()
df_forecast_new['ARCH(84)-hi-95'] = last_close_price * (1 + df_forecast_new['ARCH(84)-hi-95'] / 100).cumprod()

In [326]:
df_forecast_new

,ds,ARCH(84),ARCH(84)-lo-95,ARCH(84)-lo-90,ARCH(84)-hi-90,ARCH(84)-hi-95,unique_id,r_ARCH(84),r_ARCH(84)-lo-95,r_ARCH(84)-lo-90,r_ARCH(84)-hi-90,r_ARCH(84)-hi-95
0,2024-09-27 22:00:00+00:00,66401.867882,65341.446811,65511.934465,67291.801299,67462.288954,1,66401.867882,65341.446811,65511.934465,67291.801299,67462.288954
1,2024-09-27 23:00:00+00:00,65986.348548,63859.828920,64199.368059,67797.851840,68147.686814,1,65986.348548,63859.828920,64199.368059,67797.851840,68147.686814
2,2024-09-28 00:00:00+00:00,65911.857533,62731.551669,63235.801576,68662.355875,69197.860223,1,65911.857533,62731.551669,63235.801576,68662.355875,69197.860223
3,2024-09-28 01:00:00+00:00,65347.889636,61166.831323,61825.101619,69019.109971,69739.701980,1,65347.889636,61166.831323,61825.101619,69019.109971,69739.701980
4,2024-09-28 02:00:00+00:00,65185.584060,59984.379357,60797.389409,69823.559746,70741.453970,1,65185.584060,59984.379357,60797.389409,69823.559746,70741.453970
5,2024-09-28 03:00:00+00:00,65506.447017,59266.923192,60235.233989,71156.559438,72283.993679,1,65506.447017,59266.923192,60235.233989,71156.559438,72283.993679
6,2024-09-28 04:00:00+00:00,65084.048023,57875.152051,58985.693794,71714.991171,73049.241994,1,65084.048023,57875.152051,58985.693794,71714.991171,73049.241994
7,2024-09-28 05:00:00+00:00,64841.521326,56658.869562,57910.032524,72488.311534,74040.001686,1,64841.521326,56658.869562,57910.032524,72488.311534,74040.001686
8,2024-09-28 06:00:00+00:00,64424.030981,55319.140434,56700.922176,73068.353131,74837.285083,1,64424.030981,55319.140434,56700.922176,73068.353131,74837.285083
9,2024-09-28 07:00:00+00:00,63910.001014,53911.528603,55417.374716,73556.409124,75547.310246,1,63910.001014,53911.528603,55417.374716,73556.409124,75547.310246
